## Python socket
之前实现redis-like数据库的网络层时使用过py的socket库
像flask, django, fastapi这样的三方库很多，所以要建起一个简易的网络后台是很简单的，但这些第三方工具最核心的依赖都是socket标准库。

这个模块通过原生sockets暴露了写TCP和UDP后台所需要的接口，

## UDP
IP协议负责将数据包传输至正确的机器，而需要维护一个会话，还需要两个额外的特性。
* 需要为大量数据包打上标签，多路复用。
* 修护传输过程中遇到的任何错误，可靠传输。

UDP协议只解决了第一个问题。通过提供端口号对不同的网络会话进行隔离，而UDP协议仍然需要自己处理丢包，重包和包的乱序问题。而TCP协议还保证了数据流的顺序以及可靠传输，在接收端会对传输过来的数据包进行重组。

这一部分，讲UDP主要是为了熟悉多路复用。

计算机网络和电磁信号理论中，对共享同一通信信道的多个信号进行区分是个常见的问题。多路复用就是允许多个会话共享同一介质或机制的一种解决方案。UDP的方案是为每个UDP数据包分配一对无符号16位端口号，其范围从0~65536。

IP网络层上，唯一可见的是：
Source IP -> Destination IP

而UDP则是：
Source IP: port number -> Destination IP: port number

假设请求机对目标机进行一次请求，没有显示表明端口号，操作系统则会为该查询随机分配一个端口号。无论是windows系统还是POSIX系统，其网络操作相关的系统调用都是围绕着socket这一概念进行的。如下是一个简单的UDP服务器以及UDP客户端。

```python

# 该脚本简单定义了一个UDP脚本，请在cmd或shell下执行

import argparse, socket
from datetime import datetime

MAX_BYTES = 65535

def server(port):
    # 此处SOCK_DGRAM代表在IP网络上使用UDP协议
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    # 套接字请求绑定一个UDP网络地址
    sock.bind(('127.0.0.1', port))
    print('Listening at {}'.format(sock.getsockname()))
    while True:
        data, address = sock.recvfrom(MAX_BYTES)
        text = data.decode('ascii')
        print('The client at {} says {!r}'.format(address, text))
        text = 'Your data was {} bytes long'.format(len(data))
        data = text.encode('ascii')
        sock.sendto(data, address)

def client(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    text = 'The time is {}'.format(datetime.now())
    data = text.encode('ascii')
    sock.sendto(data, ('127.0.0.1', port))
    print('The OS assigned me the address {}'.format(sock.getsockname()))
    data, address = sock.recvfrom(MAX_BYTES)  # Danger! See Chapter 2
    text = data.decode('ascii')
    print('The server {} replied {!r}'.format(address, text))

if __name__ == '__main__':
    choices = {'client': client, 'server': server}
    parser = argparse.ArgumentParser(description='Send and receive UDP locally')
    parser.add_argument('role', choices=choices, help='which role to play')
    parser.add_argument('-p', metavar='PORT', type=int, default=1060,
                        help='UDP port (default 1060)')
    args = parser.parse_args()
    function = choices[args.role]
    function(args.p)
    
```